In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import math

total_samples = 61440

def butter_bandpass(lowcut, highcut, fs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(5, [low,high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs):
    b, a = butter_bandpass(lowcut, highcut, fs)
    y = signal.filtfilt(b, a, data)
    return y

def print_peaks(peaks):
    intervals  = 32
    for i in range(intervals):
        lo = i*1920 #1920 samples in 15 seconds
        hi = lo + 1920
        hi2 = lo + 4*1920
        result = peaks[(peaks>=lo)*(peaks<hi)]
        hr = len(result)*4

        if hi2 > total_samples:
            hrv = -1
        else:
            result = peaks[(peaks>=lo)*(peaks<hi2)]
            result = result[1:]-result[:-1]
            hrv = np.std(result)

        print("{0},{1},{2}".format(i*15,hr,hrv))
    print("")

data = np.genfromtxt('data-ecg.csv', delimiter=',')
filtered_data = butter_bandpass_filter(data,5,35,128)
peaks1, _ = signal.find_peaks(filtered_data,distance=35,height=100)
print_peaks(peaks1)
    
data = np.genfromtxt('data-ppg.csv', delimiter=',')
filtered_data = butter_bandpass_filter(data,.5,5,128)
peaks2, _ = signal.find_peaks(filtered_data, distance=35, height=100)
print_peaks(peaks2)

#task3
intervals  = 8
low_hr = 72
high_hr = 140
for i in range(intervals):
    lo = i*7680
    hi = lo + 7680
    result = peaks[(peaks>=lo)*(peaks<hi)]
    hr = len(result)
    activity_level = ((hr-low_hr)/(high_hr-low_hr))*9 + 1
    print("{0},{1}".format(i*60,math.ceil(activity_level)))
print("")



0,80,2.9210519301611573
15,80,5.570655830078101
30,76,6.299553555610112
45,80,6.486337909369555
60,84,3.3308041154296846
75,84,3.5603993297805308
90,88,3.1333611791098517
105,88,12.450208993950886
120,92,16.05408220776794
135,92,18.251704324358364
150,116,16.700281682774804
165,108,15.925052464591756
180,128,13.1889386737925
195,120,13.53471161040518
210,148,18.69318503953637
225,140,18.93435380322475
240,104,11.619888099508747
255,80,23.685868714872214
270,84,22.962034730572533
285,84,17.95292446090901
300,164,10.448837030295557
315,160,11.034695267824102
330,168,10.711674431936515
345,156,8.59265278882104
360,128,2.6006030069966464
375,128,2.4825242461304655
390,124,2.0808485181664604
405,124,2.6667667048062422
420,116,5.9985877967698
435,120,-1
450,120,-1
465,128,-1

0,76,14.414976002136893
15,84,16.90865163163521
30,76,17.74299783350752
45,76,17.656266698201662
60,88,22.150648270894052
75,80,24.40572306390242
90,88,23.73410734027445
105,84,25.867620319459917
120,100,23.114856316627